In [43]:
from newspaper import Article
import pandas as pd
import requests
import numpy as np
from datetime import datetime as dt
import time as tm
import re
from bs4 import BeautifulSoup

In [44]:
chosen = pd.read_csv('final_chosen.csv')

In [39]:
path  = 'chosenSamp/20191024_queen-of-harlem-michelle-smalls-set-to-receive-the-platinum-power-award-at-unlock-your-dreams-conference-expo-and-awards-gala.html'
f = open(path, 'rb')
article = Article('https://wayback.archive-it.org/12706/20200113151841/https://newbrunswicktoday.com/2019/06/12/under-pressure-rutgers-releases-rupd-and-nbpd-dispatch-logs/')
#article.html = str(f.read(), 'utf-8')
f.close()
article.download()
article.parse()
print(article.authors)
print(article.publish_date)
print(article.title)
#print(article.text)

['Charlie Kratovil', 'About The Writer', 'More Writer', 'November', 'On', 'In', '.Molongui-Author-Box .Molongui-Author-Box-Tabs Nav.Molongui-Author-Box-Tabs-Top Background-Color', '.Molongui-Author-Box .Molongui-Author-Box-Tabs Nav Label Background-Color', '.Molongui-Author-Box .Molongui-Author-Box-Tabs Input Id', "'Mab-Tab-Profile-'"]
2019-06-12 00:00:00
Under Pressure, Rutgers Releases RUPD and NBPD Dispatch Logs


In [37]:
chosen.date[0]

Timestamp('2019-10-08 00:00:00')

In [54]:

# this creates and sets all the new variables to None
chosen['cleaned'] = None
chosen['title'] = None
chosen['published'] = None
chosen['author'] = None
chosen['date'] = pd.to_datetime(chosen['date'])

# this function constructs the url for an archived webpage according to https://wayback.archive-it.org formatting
def get_html(collection_id, data):

    # the base url for the archive
    ai_url = "https://wayback.archive-it.org"

    # this loop iterates through all the urls, searches for the archived webpage
    # then looks for any hyperlinks on the page that are associated with a date in the list
    for index, row in data.iterrows():

        # this is the url of an archived webpage's home 
        # (e.g. https://wayback.archive-it.org/12706/*/http://mypaperonline.com)
        home = ai_url + '/' + str(collection_id) + '/*/' + row.url

        # this block gets all the date hyperlinks
        page = requests.get(home)
        soup = BeautifulSoup(page.content, "lxml")
        alist = soup.find_all("a")
        alist = [a for a in alist if 'onclick' in a.attrs]
        dates = [dt.strptime(a.text, '%b %d, %Y') for a in alist]

        # this loops through all the date hyperlinks and if they're in
        # the date_list it saves them locally
        for a in alist:
    
            if dt.strptime(a.text, '%b %d, %Y') == row.date:
                
                article = Article('https:' + a.attrs['href'])

                try:
                    data.loc[index, 'title'] = article.title
                    data.loc[index, 'author'] = article.authors[0]
                    data.loc[index, 'published'] = article.published_date
                    data.loc[index, 'text'] = article.text
                    data.loc[index, 'cleaned'] = True
                except:
                    data.loc[index, 'cleaned'] = False

    return (data)

In [57]:
start = tm.time()
new_parse = get_html(12706, chosen)
span = tm.time() - start
print('completed in ', span, ' seconds')

completed in  879.2199304103851  seconds


In [59]:
new_parse.to_csv('test_parsing.csv')